In [1]:
import torch
from modelscope import AutoModel

MODEL_NAME = "/home/public/dkx/model/BAAI/BGE-VL-v1.5-zs"

model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)
model.eval()
model.cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LLaVANextForEmbedding(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELU

In [3]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530"
)

client.load_collection("bar")

In [4]:
client.list_collections()

['bar']

In [12]:
with torch.no_grad():
    model.set_processor(MODEL_NAME)

    query_inputs = model.data_process(
        text="""
Agriculture and Food Production, Production of Different Crops In An Area, metric tons, leptokurtic, horizontal
Crop category,metric tons
Rapeseed,20000
Wheat,2694
Apple,2488
Sugarcane,7573
""",
        images=["/home/dukaixing/RAG4Ghart/Dataset-ZXQ/sample100/bar/png/21.png"],
        q_or_c="q",
        task_instruction="Recommend the most suitable chart with corresponding description for visualizing the information given by the provided text: "
    )

    query_embs = model(**query_inputs, output_hidden_states=True)[:, -1, :]

    query_embs = torch.nn.functional.normalize(query_embs, dim=-1)

    print(len(query_embs.cpu().detach().tolist()[0]))

4096


In [13]:
res = client.search(
    collection_name="bar",  # target collection
    data=query_embs.cpu().detach().tolist(),  # query vectors
    limit=10,  # number of returned entities
    output_fields=["id", "image_url", "data"],  # specifies fields to be returned
)

for x in res[0]:
    print(x["id"])
    print(x["distance"])

21
0.6780620217323303
273
0.64019376039505
911
0.6351696848869324
742
0.6291813850402832
141
0.6212880611419678
540
0.6111681461334229
454
0.6016165018081665
395
0.6001681089401245
157
0.5988056063652039
684
0.5967217683792114


In [14]:
with torch.no_grad():
    model.set_processor(MODEL_NAME)

    query_inputs1 = model.data_process(
        text="""
Agriculture and Food Production, Production of Different Crops In An Area, metric tons, leptokurtic, horizontal
Crop category,metric tons
Rapeseed,20000
Wheat,2694
Apple,2488
Sugarcane,7573
""",
        q_or_c="q",
        task_instruction="Recommend the most suitable chart with corresponding description for visualizing the information given by the provided text: "
    )

    query_embs1 = model(**query_inputs1, output_hidden_states=True)[:, -1, :]

    query_embs1 = torch.nn.functional.normalize(query_embs1, dim=-1)

    print(len(query_embs1.cpu().detach().tolist()[0]))

4096


In [15]:
res1 = client.search(
    collection_name="bar",  # target collection
    data=query_embs1.cpu().detach().tolist(),  # query vectors
    limit=10,  # number of returned entities
    output_fields=["id", "image_url", "data"],  # specifies fields to be returned
)

for x in res1[0]:
    print(x["id"])
    print(x["distance"])

911
0.3492613136768341
273
0.34827539324760437
141
0.3397336006164551
742
0.3390982151031494
84
0.3294185698032379
21
0.327197790145874
454
0.3228279948234558
540
0.31269580125808716
395
0.3067818284034729
157
0.302957147359848
